In [ ]:
import os
import json
import re # Used for advanced string manipulation/sanitization

# Define the file path for the family data
DATA_FILE = "family_data.json"

def sanitize_name(name):
    """
    Converts the name to lowercase and replaces spaces and non-word characters 
    with underscores for safe folder naming.
    
    Args:
        name (str): The raw name string.
        
    Returns:
        str: The sanitized, lowercase, and filesystem-safe name.
    """
    # 1. Convert to lowercase
    sanitized = name.lower()
    # 2. Replace spaces and consecutive non-word characters (like hyphens or periods) 
    #    with a single underscore
    sanitized = re.sub(r'[\s\W]+', '_', sanitized)
    # 3. Trim leading/trailing underscores that might result from the previous step
    return sanitized.strip('_')

def create_named_folders_from_json(data_filepath):
    """
    Reads data from a JSON file and creates folders using the format 'idNUMBER_NAME'.
    
    The JSON data is expected to be a list of objects, each containing 
    'PersonID' (the number) and 'Name' (the name part).

    Args:
        data_filepath (str): The path to the family data JSON file.
    """
    try:
        # Load and parse the JSON data
        with open(data_filepath, 'r') as f:
            family_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Data file not found at '{data_filepath}'. Please create it or check the path.")
        return
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{data_filepath}'. Check the file format for errors.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}")
        return

    if not isinstance(family_data, list):
        print("Error: JSON file content is not a list of objects. Expected a top-level list.")
        return
        
    print(f"Starting to create {len(family_data)} folders based on JSON entries...")

    folder_count = 0
    for person in family_data:
        try:
            # Safely convert PersonID to string
            person_id = str(person.get('PersonID', 'UNKNOWN'))
            raw_name = person.get('Name', 'unknown_name')
            
            # Sanitize the name for folder creation
            folder_name_part = sanitize_name(raw_name)
            
            # Construct the final folder name: idNUMBER_NAME
            folder_name = f"id{person_id}_{folder_name_part}"
            
            # Skip if the name part is empty after sanitization
            if not folder_name_part:
                print(f"Warning: Skipping ID {person_id} due to empty or invalid name data.")
                continue

            # Create the directory
            os.makedirs(folder_name, exist_ok=True)
            print(f"Successfully created: {folder_name}")
            folder_count += 1
            
        except OSError as e:
            # Handle system errors (like permissions)
            print(f"Error creating directory {folder_name}: {e}")
        except Exception as e:
            # Handle other unforeseen errors in the loop
            print(f"An unexpected error occurred processing an entry: {e}")
            
    print(f"\nFinished. Total folders created: {folder_count}")


if __name__ == "__main__":
    # The data file path is defined globally at the top of the script
    create_named_folders_from_json(DATA_FILE)


Starting to create 55 folders based on JSON entries...
Successfully created: id1_misha
Successfully created: id2_noname
Successfully created: id3_sergey
Successfully created: id4_dima
Successfully created: id5_max
Successfully created: id6_sophia
Successfully created: id7_sviatik
Successfully created: id8_nadia
Successfully created: id9_fedia
Successfully created: id10_nikolay
Successfully created: id11_melania
Successfully created: id12_gleb
Successfully created: id13_masha
Successfully created: id14_olga
Successfully created: id15_dima
Successfully created: id16_anna
Successfully created: id17_igor
Successfully created: id18_sergey
Successfully created: id19_vika
Successfully created: id20_nikolay
Successfully created: id21_leonid
Successfully created: id22_elena
Successfully created: id23_sasha
Successfully created: id24_anton
Successfully created: id25_unia
Successfully created: id26_elina
Successfully created: id27_appolinar
Successfully created: id28_babush_belar
Successfully cre